In [45]:
import pandas as pd
import numpy as np

import csv

import re

import itertools

import matplotlib.pyplot as plt
import mpld3
import scipy.stats as stats

Simple rating calculator (doesn't join any databases) 

by Bobby Mills and Arseny Khakhalin

In [29]:
def fixName(name):
    res = re.sub(' ', '_', name)
    res = re.sub(',_', '_', res)
    res = re.sub('_\d+', '', res) #remove _number from lands
    res = re.sub('_\([a-zA-Z]\)', '', res) #remove guildgate types
    res = res.lower()
    return res

In [30]:
# Change the data file below to work with the draftsim data you want

draftData = pd.read_csv('../2018-11-29 UMA.csv', 
                       names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])

#draftData = pd.read_csv('../2018-10-05 GRN Draft Data 1.csv', 
#                       names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('05-31-18 Rows 500000-600000.csv', 
#                       names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('../2018-08-23 m19 drafts round 2.csv', 
#                      names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
#draftData = pd.read_csv('2018-04-16 Dominiaria initial data-2.csv', 
#                        names = ['format', 'human', 'bot1', 'bot2', 'bot3', 'bot4', 'bot5', 'bot6', 'bot7'])
draftData.head()

,format,human,bot1,bot2,bot3,bot4,bot5,bot6,bot7
2538900,UMA,"Seize_the_Day,Nourishing_Shoal,Pattern_of_Rebi...","Become_Immense,Gurmag_Angler,Moan_of_the_Unhal...","Soul's_Fire,Reckless_Wurm,Wild_Mongrel,Aethers...","Reveillark,Hero_of_Iroas,Icatian_Crier,Convict...","Faith's_Fetters,Boar_Umbra,Wingsteed_Rider,Pat...","Talrand,_Sky_Summoner,Aethersnipe,Satyr_Wayfin...","Treasure_Cruise,Fiery_Temper,Cathodion,Rogue's...","Fiery_Temper,Reckless_Charge,Reckless_Charge,M..."
2538914,UMA,"Rune_Snag,Plumeveil,Scuzzback_Marauders,Prey_U...","Seize_the_Day,Iridescent_Drake,Fulminator_Mage...","Last_Gasp,Moan_of_the_Unhallowed,Spider_Spawni...","Wall_of_Reverence,Reckless_Charge,Warleader's_...","Bitterblossom,Golgari_Thug,Buried_Alive,Ghoulc...","Sigil_of_the_New_Dawn,Moan_of_the_Unhallowed,W...","Moan_of_the_Unhallowed,Penumbra_Wurm,Become_Im...","Last_Gasp,Death_Denied,Pattern_of_Rebirth,Cath..."
2538932,UMA,"Bridge_from_Below,Marang_River_Prowler,Garna,_...","Eternal_Witness,Faith's_Fetters,Penumbra_Wurm,...","Penumbra_Wurm,Seize_the_Day,Wild_Mongrel,Wild_...","Fiery_Temper,Aethersnipe,Wingsteed_Rider,Shire...","Wingsteed_Rider,Soul's_Fire,Dimir_Guildmage,Go...","Brazen_Scourge,Fire_Ice,Reckless_Charge,Gamble...","Wall_of_Reverence,Hero_of_Iroas,Daybreak_Coron...","Golgari_Grave-Troll,Fiery_Temper,Wild_Mongrel,..."
2538961,UMA,"Faith's_Fetters,Warleader's_Helix,Wandering_Ch...","Eternal_Witness,Fiery_Temper,Kodama's_Reach,Sa...","Wingsteed_Rider,Fiery_Temper,Firewing_Phoenix,...","Talrand,_Sky_Summoner,Marang_River_Prowler,Las...","Lavaclaw_Reaches,Faith's_Fetters,Cathodion,Ter...","Wingsteed_Rider,Devoted_Druid,Sigil_of_the_New...","Maelstrom_Pulse,Death_Denied,Moan_of_the_Unhal...","Faith's_Fetters,Eternal_Witness,Daybreak_Coron..."
2538962,UMA,"Garna,_the_Bloodflame,Unburial_Rites,Soul's_Fi...","Fiery_Temper,Cathodion,Reckless_Wurm,Brazen_Sc...","Kitchen_Finks,Brawn,Wickerbough_Elder,Wingstee...","Treasure_Cruise,Firewing_Phoenix,Fire_Ice,Dera...","Noble_Hierarch,Thermo-Alchemist,Boar_Umbra,Bon...","Snapcaster_Mage,Fire_Ice,Fire_Ice,Archaeomance...","Wall_of_Reverence,Last_Gasp,Hero_of_Iroas,Mira...","Sublime_Archangel,Sigil_of_the_New_Dawn,Gurmag..."


In [32]:
# Pick order variable
cardpicks = {}

count = 0
player = 'human' # normally should be: human, but can also be bot1 or another bot
for draft in draftData[player]:
    draft = fixName(draft)
    draft = draft.lower()
    draft = draft.split(',')
    count = count+1    
    for i in range(14):
        try:
            if draft[i] in cardpicks:
                cardpicks[draft[i]].append(i+1)
            else:
                cardpicks[draft[i]] = [i+1]       
        except KeyError as e:
            print(draft)
            print(count)
            raise

In [33]:
for card in cardpicks:
    if cardpicks[card] == []:
        cardpicks[card].append(15)

In [34]:
cardpicksdf = pd.DataFrame({
        'avg' : [np.mean(cardpicks[card]) for card in cardpicks],
        'var' : [np.var(cardpicks[card]) for card in cardpicks],
        'count' : [len(cardpicks[card]) for card in cardpicks]
    }, list(cardpicks.keys()))

In [35]:
cardpicksdf = cardpicksdf.sort_values(by=['avg'], ascending=True)

In [75]:
cp2 = cardpicksdf.copy()
cp2['cohen_d'] = (cp2['avg']-cp2['avg'][0])/np.sqrt((cp2['var']+cp2['var'][0])/2)
cp2['n_adj'] = np.sqrt(1/(1/cp2['count']+ 1/cp2['count'][0]))
cp2['p-value'] = 1-stats.t.cdf(cp2['cohen_d']*cp2['n_adj'],cp2['n_adj']-1)

# Justificatin for the formula: http://imaging.mrc-cbu.cam.ac.uk/statswiki/FAQ/td
# apparently based on:
# Pustejovsky, J. E. (2014) Converting from d to r to z when the design uses extreme groups, 
# dichotomization, or experimental control. Psychological Methods 19(1) 92-112.

cp2.head(20)

,avg,var,count,cohen_d,n_adj,p-value
bitterblossom,1.002660,0.002653,376,0.000000,13.711309,0.500000
mikaeus_the_unhallowed,1.002817,0.002809,355,0.003011,13.512926,0.484095
balefire_dragon,1.003135,0.003125,319,0.008842,13.137020,0.454715
karn_liberated,1.004902,0.004878,408,0.036544,13.988333,0.308899
mana_vault,1.006116,0.006079,327,0.052315,13.224831,0.250977
sublime_archangel,1.007407,0.007353,945,0.067128,16.400550,0.143919
snapcaster_mage,1.007481,0.007425,401,0.067926,13.930144,0.180694
liliana_of_the_veil,1.008180,0.008113,489,0.075243,14.579422,0.145857
demonic_tutor,1.011236,0.011110,801,0.103389,15.996389,0.059466
golgari_grave-troll,1.011412,0.011282,701,0.104860,15.643902,0.061107


In [76]:
cp2.to_csv('../simple_rating.csv', index_label="name")